# ILP and Seq-Greedy Compare

The two are leading to very different results given ample resources.
We will assume that dijkstra's algorithm works properly in the SEQ greedy case, and examine a simple scenario and see if the weights line up.

In [1]:
# Import Generic Classes
import numpy as np
import copy

# Import All Custom Classes
import os, sys
sys.path.append(os.path.pardir+"/classes")

from Server import *
from User import *
from Link import *
from Job import *
from Migration_Plans import *


# Import Solver Classes
from Optim_PlanGenerator import *
from SeqGreedy_PlanGenerator import*

In [68]:
"""
Make Simulation Parameters
"""
sim_param = Sim_Params(time_steps=3, x_length = 5, y_length = 5, max_edge_length=2)
boundaries = np.array([[0,sim_param.x_length],[0,sim_param.y_length]])


"""
Make Job Profiles
"""
# REsources used are CPU (no. cores) storage (GB), and RAM (GB)
# througput is in mb/s
# Latency is in ms

job_profile1 = Job_Profile(job_name = "VR",
                           latency_req_range=[0, 0], 
                           thruput_req_range=[1, 1], 
                           length_range=[3,3],  
                           placement_rsrc_range = np.array([[1,1]]),
                           migration_amt_range = [1, 1],
                           latency_penalty_range = [1,1])#[0.05, 0.1]) 


job_profiles = [job_profile1]


"""
Make Servers
"""

# Server Settings
num_server_l1 = 0
num_server_l2 = 2
num_server_l3 = 0

num_resource = 1
weak_range = np.array([[1000,1500]])
strong_range = np.array([[100000,150000]])

rsrc_cost = np.array([1])

# Generate Server
servers_l1 = []
servers_l2 = []
servers_l3 = []
idx_counter = 0

for i in range(num_server_l2):
    servers_l2.append(Server(boundaries,level=2,rand_locs=True,locs=None))
    servers_l2[-1].server_resources(num_resource, weak_range, strong_range)
    servers_l2[-1].assign_id(idx_counter)
    servers_l2[-1].server_resources_cost(num_resource,rsrc_cost)
    idx_counter += 1
    
    
servers = servers_l2 


"""
Make Links
"""

# Link Settings
num_link = [0,1,2]
prob_link = [0,1,0]
lv_minmax = np.array(([[500,1000],[10000,20000],[30000,50000]]))
lv1_transmission = 1
link_costs = np.array([1, 1, 1])
latency_settings = [1, 1] #[ms per switch, ms per mile]

links = Link(servers, num_link, prob_link, lv_minmax, link_costs, latency_settings,lv1_transmission)


"""
Make Users
"""

# User Settings
num_user_m0 = 0 # Pedestrian
num_user_m1 = 0 # Public Transport
num_user_m2 = 1 # Vehicle

max_speed = 2.5
lamdas = [1/0.25,1/0.83,1/1.67] # 3 mph, 10 mph, 20 mph
num_path = 10

# Generate Server
users_m0 = []
users_m1 = []
users_m2 = []
idx_counter = 0


for i in range(num_user_m2):
    users_m2 += [User(boundaries, sim_param.time_steps, 2, lamdas, max_speed, num_path)]
    users_m2[-1].generate_MC(servers)
    users_m2[-1].assign_id(idx_counter)
    idx_counter += 1

users = users_m2
    
    
"""
Make Jobs
- "I'm just going to do it"
"""

# Job settings
job_type0 = 1 # VR


jobs0 = []
idx_counter = 0

total_job_count = job_type0
draw_job_id = np.random.choice(total_job_count, total_job_count, replace=False)

for i in range(job_type0):
    jobs0 += [Job(job_type = 0,
                  user_id = draw_job_id[idx_counter],
                  time_steps=sim_param.time_steps,
                  job_profiles = job_profiles)]
    idx_counter += 1
    
jobs = jobs0 

In [69]:
# Edit User Voronoi Characteristics
users[0].server_prob = np.array([[1,1,0],[0,0,1]])
users[0].user_voronoi_true = np.array([0,0,1])

In [70]:
# Make ILP and Optim Prob
optim_prob = Optim_PlanGenerator(users, servers, links, jobs, sim_param)
optim_prob.prob.solve()
print("Status:", constants.LpStatus[optim_prob.prob.status])

Status: Optimal


In [71]:
ILP_mig_plan = Migration_Plans(users, jobs, sim_param) 
ILP_mig_plan.from_ILP(optim_prob)

In [72]:
ILP_mig_plan.mig_plan_dict[0]

{'time_slot': array([0., 1., 2.]),
 'user_active_flag': array([1., 1., 1.]),
 'user_voronoi': array([0., 0., 1.]),
 'source_server': array([0., 0., 1.]),
 'dest_server': array([0., 1., 1.]),
 'mig_rate': array([0., 1., 0.]),
 'mig_link_id': array([0., 0., 0.]),
 'service_link_id': array([0., 0., 0.]),
 'service_thruput': array([0., 0., 0.]),
 'latency': array([0., 0., 0.])}

In [73]:
# Seq Greedy
SG_prob = SeqGreedy_PlanGenerator(users, servers, links, jobs, sim_param)
SG_prob.calc_all_costs(j=0)
SG_prob.obtain_minimum_cost_j(j=0)

In [74]:
node_num, link_num = SG_prob.dijkstra_j(j=0,start_node=0,end_node=7)
SG_mig_plan = Migration_Plans(users, jobs, sim_param) 
SG_mig_plan.from_seq_greedy(SG_prob)

In [75]:
SG_mig_plan.seq_greedy_plan_extract(node_orders=node_num, link_path_orders=link_num, job_num=0)

In [76]:
SG_mig_plan.mig_plan_dict[0]

{'time_slot': array([0., 1., 2.]),
 'user_active_flag': array([1., 1., 1.]),
 'user_voronoi': array([0., 0., 1.]),
 'source_server': array([0., 0., 1.]),
 'dest_server': array([0., 1., 1.]),
 'mig_rate': array([0., 1., 0.]),
 'mig_link_id': array([0., 0., 0.]),
 'service_link_id': array([0., 0., 0.]),
 'service_thruput': array([0., 0., 0.]),
 'latency': array([0., 0., 0.])}

In [77]:
SG_prob.valid_links

[array([[0., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1., 1., 1., 0., 0., 1., 0.],
        [0., 0., 0., 0., 1., 1., 1., 0., 1., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 0.],
        [0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])]

In [78]:
print(SG_prob.edge_weights_min[0])

[[ 0.          1.          1.          4.26559354  0.          0.
   0.          0.          0.          0.          0.        ]
 [ 0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.        ]
 [ 0.          0.          0.          0.          1.          1.
   6.26559354  0.          0.          5.          0.        ]
 [ 0.          0.          0.          0.          1.          3.
   4.26559354  0.         11.53118708  0.          0.        ]
 [ 0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.        ]
 [ 0.          0.          0.          0.          0.          0.
   0.          1.          4.26559354  3.          0.        ]
 [ 0.          0.          0.          0.          0.          0.
   0.          1.          6.26559354  1.          0.        ]
 [ 0.          0.          0.          0.          0.          0.
   0.          0.          0.  

In [79]:
SG_prob = SeqGreedy_PlanGenerator(users, servers, links, jobs, sim_param)

SG_prob.convert_st2node

[{(-1, -1): 0,
  (-1, 0): 1,
  (0, 0): 2,
  (1, 0): 3,
  (-1, 1): 4,
  (0, 1): 5,
  (1, 1): 6,
  (-1, 2): 7,
  (0, 2): 8,
  (1, 2): 9,
  (-1, 3): 10}]

In [80]:
SG_prob.edge_weights_min

{}

In [81]:
for k in SG_prob.resource_constraints.server_rsrc[0,:,0]:
    print(k)

133456.9049238265


In [59]:
SG_prob.resource_constraints.server_rsrc[0,:,0].shape[0]

2

In [82]:
links.get_subpath_sequential(0,1,0)

(0, 1)

In [88]:
ca = np.array([[0,1],[1,1]])
np.all(ca >= 0)

True